In [48]:
import re
import pandas as pd
from flair.data import Sentence
from flair.models import TextClassifier
Flair_Classifier=TextClassifier.load('en-sentiment')
from transformers import pipeline
classifierHG=pipeline('sentiment-analysis')

2022-06-06 16:22:49,558 loading file /Users/aniltilbe/.flair/models/sentiment-en-mix-distillbert_4.pt


In [49]:
# https://www.kaggle.com/datasets/snap/amazon-fine-food-reviews. Download it here

Reviews=pd.read_csv('/Users/aniltilbe/Jupyter Notebooks/Training/Reviews.csv')
Reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568454 entries, 0 to 568453
Data columns (total 10 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   Id                      568454 non-null  int64 
 1   ProductId               568454 non-null  object
 2   UserId                  568454 non-null  object
 3   ProfileName             568438 non-null  object
 4   HelpfulnessNumerator    568454 non-null  int64 
 5   HelpfulnessDenominator  568454 non-null  int64 
 6   Score                   568454 non-null  int64 
 7   Time                    568454 non-null  int64 
 8   Summary                 568427 non-null  object
 9   Text                    568454 non-null  object
dtypes: int64(5), object(5)
memory usage: 43.4+ MB


In [50]:
#Isolate 10 rows for experiementation and drop rows

new_result=Reviews.iloc[19:29]
new_result=new_result.drop(columns=["ProductId","UserId","ProfileName","Summary","HelpfulnessNumerator",\
                                    "HelpfulnessDenominator","Time"])
new_result

,Id,Score,Text
19,20,5,Candy was delivered very fast and was purchase...
20,21,5,My husband is a Twizzlers addict. We've bough...
21,22,5,I bought these for my husband who is currently...
22,23,5,I can remember buying this candy as a kid and ...
23,24,5,I love this candy. After weight watchers I ha...
24,25,5,"I have lived out of the US for over 7 yrs now,..."
25,26,5,Product received is as advertised.<br /><br />...
26,27,1,"The candy is just red , No flavor . Just plan..."
27,28,4,I was so glad Amazon carried these batteries. ...
28,29,5,I got this for my Mum who is not diabetic but ...


In [51]:
#Print Flair prediction and confidence score in two new columns 

def sentiment_Flair(x_score):
    sentence3=Sentence(x_score)
    Flair_Classifier.predict(sentence3)
    label8=sentence3.labels[0].value
    
    if "POSITIVE" in str(label8):
        return 'Positive'
    else:
        return 'Negative'
new_result['Flair Prediction']=new_result['Text'].apply(lambda x_score:sentiment_Flair(x_score))


def sentiment_Flair(x_label):
    sentence2=Sentence(x_label)
    Flair_Classifier.predict(sentence2)
    score8=sentence2.labels[0].score
    
    if "POSITIVE" in str(sentence2):
        return score8
    else: 
        return score8
new_result['Flair Confidence Score']=new_result['Text'].apply(lambda x_label:sentiment_Flair(x_label))



In [52]:
new_result

,Id,Score,Text,Flair Prediction,Flair Confidence Score
19,20,5,Candy was delivered very fast and was purchase...,Positive,0.999845
20,21,5,My husband is a Twizzlers addict. We've bough...,Positive,0.998997
21,22,5,I bought these for my husband who is currently...,Positive,0.999948
22,23,5,I can remember buying this candy as a kid and ...,Positive,0.998611
23,24,5,I love this candy. After weight watchers I ha...,Positive,0.999849
24,25,5,"I have lived out of the US for over 7 yrs now,...",Positive,0.999723
25,26,5,Product received is as advertised.<br /><br />...,Positive,0.998591
26,27,1,"The candy is just red , No flavor . Just plan...",Negative,0.999993
27,28,4,I was so glad Amazon carried these batteries. ...,Positive,0.999930
28,29,5,I got this for my Mum who is not diabetic but ...,Positive,0.999680


In [53]:
#Print HuggingFace prediction and confidence score in two new columns

def transformer_sentiment(sentenceHG_Label):
    sentencesLabel_sentiment=classifierHG(sentenceHG_Label)
    if "POSITIVE" in str(sentencesLabel_sentiment):
        return "Positive"
    else:
        return "Negative"
    
new_result['HuggingFace Predictions']=new_result["Text"].apply(lambda sentenceHG_Label: \
                                                               transformer_sentiment(sentenceHG_Label))
def transformer_sentiment(sentenceHG_Score):
    sentencesScore_sentiment=classifierHG(sentenceHG_Score)
    decimal_output=re.findall('\d*\.?\d+', str(sentencesScore_sentiment)) #pull the #'s
    if "POSITIVE" in str(sentencesScore_sentiment):
        return decimal_output
    else:
        return decimal_output
    
new_result['HuggingFace Score']=new_result["Text"].apply(lambda sentenceHG_Score: \
                                                          transformer_sentiment(sentenceHG_Score))

In [54]:
new_result

,Id,Score,Text,Flair Prediction,Flair Confidence Score,HuggingFace Predictions,HuggingFace Score
19,20,5,Candy was delivered very fast and was purchase...,Positive,0.999845,Positive,[0.9976369142532349]
20,21,5,My husband is a Twizzlers addict. We've bough...,Positive,0.998997,Positive,[0.9998146891593933]
21,22,5,I bought these for my husband who is currently...,Positive,0.999948,Positive,[0.9996742010116577]
22,23,5,I can remember buying this candy as a kid and ...,Positive,0.998611,Positive,[0.9995959401130676]
23,24,5,I love this candy. After weight watchers I ha...,Positive,0.999849,Positive,[0.9991275668144226]
24,25,5,"I have lived out of the US for over 7 yrs now,...",Positive,0.999723,Positive,[0.9971465468406677]
25,26,5,Product received is as advertised.<br /><br />...,Positive,0.998591,Negative,[0.9962425231933594]
26,27,1,"The candy is just red , No flavor . Just plan...",Negative,0.999993,Negative,[0.9988172650337219]
27,28,4,I was so glad Amazon carried these batteries. ...,Positive,0.999930,Negative,[0.5259760618209839]
28,29,5,I got this for my Mum who is not diabetic but ...,Positive,0.999680,Positive,[0.9997897744178772]


In [55]:
new_result=new_result[["Id","Score","Text","Flair Prediction","HuggingFace Predictions", \
                      "Flair Confidence Score","HuggingFace Score"]]
new_result

,Id,Score,Text,Flair Prediction,HuggingFace Predictions,Flair Confidence Score,HuggingFace Score
19,20,5,Candy was delivered very fast and was purchase...,Positive,Positive,0.999845,[0.9976369142532349]
20,21,5,My husband is a Twizzlers addict. We've bough...,Positive,Positive,0.998997,[0.9998146891593933]
21,22,5,I bought these for my husband who is currently...,Positive,Positive,0.999948,[0.9996742010116577]
22,23,5,I can remember buying this candy as a kid and ...,Positive,Positive,0.998611,[0.9995959401130676]
23,24,5,I love this candy. After weight watchers I ha...,Positive,Positive,0.999849,[0.9991275668144226]
24,25,5,"I have lived out of the US for over 7 yrs now,...",Positive,Positive,0.999723,[0.9971465468406677]
25,26,5,Product received is as advertised.<br /><br />...,Positive,Negative,0.998591,[0.9962425231933594]
26,27,1,"The candy is just red , No flavor . Just plan...",Negative,Negative,0.999993,[0.9988172650337219]
27,28,4,I was so glad Amazon carried these batteries. ...,Positive,Negative,0.999930,[0.5259760618209839]
28,29,5,I got this for my Mum who is not diabetic but ...,Positive,Positive,0.999680,[0.9997897744178772]


In [58]:
print(new_result[new_result["Id"]==28])

    Id  Score                                               Text  \
27  28      4  I was so glad Amazon carried these batteries. ...   

   Flair Prediction HuggingFace Predictions  Flair Confidence Score  \
27         Positive                Negative                 0.99993   

       HuggingFace Score  
27  [0.5259760618209839]  
